In [28]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sklearn.preprocessing import OneHotEncoder
from category_encoders import LeaveOneOutEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, average_precision_score, precision_recall_curve
import lightgbm as lgb

In [29]:
#DATABASE & DATA LOADING
load_dotenv("C:/Users/daivi/Desktop/bank-credit-risk-platform/.env.txt", override=True) 

engine = create_engine(
    f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@"
    f"{os.getenv('PG_HOST')}:{os.getenv('PG_PORT')}/{os.getenv('PG_DB')}"
)

df = pd.read_sql("SELECT * FROM loan_applications;", engine)
df.head(10)

,id,loan_amount,funded_amount,funded_amount_investor,term,batch_enrolled,interest_rate,grade,sub_grade,employment_duration,...,recoveries,collection_recovery_fee,collection_12_months_medical,application_type,last_week_pay,accounts_delinquent,total_collection_amount,total_current_balance,total_revolving_credit_limit,loan_status
0,65087372,10000,32236,12329.362860,59,BAT2522922,11.135007,B,C4,MORTGAGE,...,2.498291,0.793724,0,INDIVIDUAL,49,0,31,311301,6619,0
1,1450153,3609,11940,12191.996920,59,BAT1586599,12.237563,C,D3,RENT,...,2.377215,0.974821,0,INDIVIDUAL,109,0,53,182610,20885,0
2,1969101,28276,9311,21603.224550,59,BAT2136391,12.545884,F,D4,MORTGAGE,...,4.316277,1.020075,0,INDIVIDUAL,66,0,34,89801,26155,0
3,6651430,11170,6954,17877.155850,59,BAT2428731,16.731201,C,C3,MORTGAGE,...,0.107020,0.749971,0,INDIVIDUAL,39,0,40,9189,60214,0
4,14354669,16890,13226,13539.926670,59,BAT5341619,15.008300,C,D4,MORTGAGE,...,1294.818751,0.368953,0,INDIVIDUAL,18,0,430,126029,22579,0
5,50509046,34631,30203,8635.931613,36,BAT4694572,17.246986,B,G5,RENT,...,5.043575,0.581688,0,INDIVIDUAL,32,0,42,51252,27480,0
6,32737431,30844,19773,15777.511830,59,BAT4808022,10.731432,C,C5,RENT,...,3.167937,0.553076,0,INDIVIDUAL,71,0,3388,42069,31068,0
7,63151650,20744,10609,7645.014802,58,BAT2558388,13.993688,A,A5,OWN,...,0.098448,0.047589,0,INDIVIDUAL,87,0,48,184909,43303,0
8,4279662,9299,11238,13429.456610,59,BAT5341619,11.178457,G,C2,MORTGAGE,...,0.530214,0.216985,0,INDIVIDUAL,144,0,26,68126,7482,0
9,4431034,19232,8962,7004.097481,58,BAT2078974,5.520413,C,B5,RENT,...,2.912215,0.886864,0,INDIVIDUAL,9,0,35,71650,14871,0


In [30]:
# Dropping unnecessary columns
cols_to_drop = [
    "total_received_interest", "total_received_late_fee", "recoveries",
    "collection_recovery_fee", "collection_12_months_medical", "last_week_pay",
    "total_collection_amount", "total_current_balance", "total_revolving_credit_limit",
    "payment_plan", "accounts_delinquent"
]
df.drop(columns=cols_to_drop, inplace=True)
df.shape

(67463, 24)

In [32]:
# Manual feature engineering
# One-Hot Encoding
ohe_cols = ['initial_list_status', 'application_type', 'verification_status', 'employment_duration']
encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_array = encoder.fit_transform(df[ohe_cols])
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(ohe_cols), index=df.index)

df_final = pd.concat([df.drop(columns=ohe_cols), encoded_df], axis=1)
df_final.head(10)

,id,loan_amount,funded_amount,funded_amount_investor,term,batch_enrolled,interest_rate,grade,sub_grade,home_ownership,...,revolving_balance,revolving_utilities,total_accounts,loan_status,initial_list_status_w,application_type_JOINT,verification_status_Source Verified,verification_status_Verified,employment_duration_OWN,employment_duration_RENT
0,65087372,10000,32236,12329.362860,59,BAT2522922,11.135007,B,C4,176346.62670,...,24246,74.932551,7,0,1.0,0.0,0.0,0.0,0.0,0.0
1,1450153,3609,11940,12191.996920,59,BAT1586599,12.237563,C,D3,39833.92100,...,812,78.297186,13,0,0.0,0.0,1.0,0.0,0.0,1.0
2,1969101,28276,9311,21603.224550,59,BAT2136391,12.545884,F,D4,91506.69105,...,1843,2.073040,20,0,1.0,0.0,1.0,0.0,0.0,0.0
3,6651430,11170,6954,17877.155850,59,BAT2428731,16.731201,C,C3,108286.57590,...,13819,67.467951,12,0,1.0,0.0,1.0,0.0,0.0,0.0
4,14354669,16890,13226,13539.926670,59,BAT5341619,15.008300,C,D4,44234.82545,...,1544,85.250761,22,0,1.0,0.0,1.0,0.0,0.0,0.0
5,50509046,34631,30203,8635.931613,36,BAT4694572,17.246986,B,G5,98957.47561,...,2277,51.564476,20,0,1.0,0.0,0.0,0.0,0.0,1.0
6,32737431,30844,19773,15777.511830,59,BAT4808022,10.731432,C,C5,102391.82430,...,14501,46.808804,37,0,1.0,0.0,0.0,1.0,0.0,1.0
7,63151650,20744,10609,7645.014802,58,BAT2558388,13.993688,A,A5,61723.52014,...,13067,23.936624,33,0,1.0,0.0,0.0,0.0,1.0,0.0
8,4279662,9299,11238,13429.456610,59,BAT5341619,11.178457,G,C2,63205.09072,...,549,15.947386,17,0,1.0,0.0,0.0,1.0,0.0,0.0
9,4431034,19232,8962,7004.097481,58,BAT2078974,5.520413,C,B5,42015.46586,...,1361,35.073345,30,0,0.0,0.0,1.0,0.0,0.0,1.0


In [33]:
# Ordinal encoding (Grade & Sub-Grade)
grade_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7}
sub_grades_sorted = sorted(df_final['sub_grade'].unique())
sub_grade_map = {val: i+1 for i, val in enumerate(sub_grades_sorted)}

df_final['grade'] = df_final['grade'].map(grade_map)
df_final['sub_grade'] = df_final['sub_grade'].map(sub_grade_map)
df_final.head(5)

,id,loan_amount,funded_amount,funded_amount_investor,term,batch_enrolled,interest_rate,grade,sub_grade,home_ownership,...,revolving_balance,revolving_utilities,total_accounts,loan_status,initial_list_status_w,application_type_JOINT,verification_status_Source Verified,verification_status_Verified,employment_duration_OWN,employment_duration_RENT
0,65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,2,14,176346.62670,...,24246,74.932551,7,0,1.0,0.0,0.0,0.0,0.0,0.0
1,1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,3,18,39833.92100,...,812,78.297186,13,0,0.0,0.0,1.0,0.0,0.0,1.0
2,1969101,28276,9311,21603.22455,59,BAT2136391,12.545884,6,19,91506.69105,...,1843,2.073040,20,0,1.0,0.0,1.0,0.0,0.0,0.0
3,6651430,11170,6954,17877.15585,59,BAT2428731,16.731201,3,13,108286.57590,...,13819,67.467951,12,0,1.0,0.0,1.0,0.0,0.0,0.0
4,14354669,16890,13226,13539.92667,59,BAT5341619,15.008300,3,19,44234.82545,...,1544,85.250761,22,0,1.0,0.0,1.0,0.0,0.0,0.0


In [34]:
# Loan title cleaning
df_final['loan_title'] = df_final['loan_title'].str.lower().str.strip()
title_mapping = {
    'debt consolidation loan': 'debt consolidation', 'consolidation loan': 'debt consolidation',
    'consolidation': 'debt consolidation', 'consolidate': 'debt consolidation',
    'credit card consolidation': 'credit card refinancing', 'credit card refinance': 'credit card refinancing',
    'credit card payoff': 'credit card refinancing', 'credit card pay off': 'credit card refinancing',
    'payoff': 'credit card refinancing', 'cc refi': 'credit card refinancing',
    'credit card refinance loan': 'credit card refinancing'
}
df_final['loan_title'] = df_final['loan_title'].replace(title_mapping)

In [35]:
# Stratified train-test split
X = df_final.drop(columns=['loan_status', 'id'])
y = df_final['loan_status']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [36]:
# High cardinality endoing (LOO)- Preventing Leakage
loo_encoder = LeaveOneOutEncoder(cols=['loan_title', 'batch_enrolled'])
X_train = loo_encoder.fit_transform(X_train, y_train)
X_test = loo_encoder.transform(X_test)

# Ensure all data is numeric for LightGBM
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

In [37]:
# LightGBM Training
# Calculate weight for the minority class (Defaulters)
imbalance_ratio = (y_train == 0).sum() / (y_train == 1).sum()

lgbm_model = lgb.LGBMClassifier(
    n_estimators=2000,
    learning_rate=0.03,      # Slightly lower for better generalization
    max_depth=7,
    num_leaves=64,           # Increased slightly for tree complexity
    scale_pos_weight=imbalance_ratio,
    objective='binary',
    metric='binary_logloss',
    random_state=42,
    verbosity=-1
)

lgbm_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(100)]
)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.50833
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.308356


LGBMClassifier(learning_rate=0.03, max_depth=7, metric='binary_logloss',
               n_estimators=2000, num_leaves=64, objective='binary',
               random_state=42, scale_pos_weight=np.float64(9.80913278590026),
               verbosity=-1)

In [39]:
# Probability prediction and evaluation
y_prob = lgbm_model.predict_proba(X_test)[:, 1]

print(f"\nFinal LightGBM Log Loss: {log_loss(y_test, y_prob):.4f}")
print(f"Final PR-AUC Score: {average_precision_score(y_test, y_prob):.4f}")


Final LightGBM Log Loss: 0.3084
Final PR-AUC Score: 0.0924


In [45]:
results_df = results_df.copy()

results_df['risk_percentile'] = results_df['predicted_pd'].rank(
    pct=True,
    method='average'
)

def assign_bucket(p):
    if p >= 0.80:
        return 'High Risk'
    elif p >= 0.50:
        return 'Medium Risk'
    else:
        return 'Low Risk'

results_df['risk_bucket_pct'] = results_df['risk_percentile'].apply(assign_bucket)

bucket_summary_pct = (
    results_df
    .groupby('risk_bucket_pct')
    .agg(
        loans=('actual_default', 'count'),
        defaults=('actual_default', 'sum'),
        default_rate=('actual_default', 'mean')
    )
    .reset_index()
)

bucket_summary_pct['default_rate'] = bucket_summary_pct['default_rate'].round(4)
print(bucket_summary_pct)

  risk_bucket_pct  loans  defaults  default_rate
0       High Risk    337        30        0.0890
1        Low Risk  13156      1218        0.0926


In [46]:
powerbi_df = pd.DataFrame({
    "id": X_test.index,              # or use stored id column if you kept it
    "predicted_pd": results_df["predicted_pd"],
    "risk_percentile": results_df["risk_percentile"],
    "risk_bucket": results_df["risk_bucket_pct"],
    "actual_default": results_df["actual_default"]
})

In [47]:
powerbi_df["grade"] = df.loc[X_test.index, "grade"]
powerbi_df["interest_rate"] = df.loc[X_test.index, "interest_rate"]
powerbi_df["loan_amount"] = df.loc[X_test.index, "loan_amount"]

In [48]:
print(powerbi_df.head())
print(powerbi_df.shape)

          id  predicted_pd  risk_percentile risk_bucket  actual_default grade  \
59394  59394      0.089776         0.487549    Low Risk               1     C   
66511  66511      0.089776         0.487549    Low Risk               0     D   
2078    2078      0.089776         0.487549    Low Risk               0     C   
29534  29534      0.089776         0.487549    Low Risk               0     C   
45950  45950      0.089776         0.487549    Low Risk               0     B   

       interest_rate  loan_amount  
59394      10.434380        20825  
66511      11.452818        32077  
2078       10.017116        30296  
29534       8.021259        31349  
45950      10.460078        32385  
(13493, 8)


In [49]:
powerbi_df.to_sql(
    "loan_risk_scoring_output",
    engine,
    if_exists="replace",
    index=False
)

493